In [1]:
import tensorflow as tf
import numpy as np

## 하이퍼 파라미터

In [2]:
EPOCHS = 10

## Residual Unit 구현

In [3]:
class ResidualUnit(tf.keras.Model):
    def __init__(self, filter_in, filter_out, kernel_size):
        super(ResidualUnit, self).__init__()
        self.bn1 = tf.keras.layers.BatchNormalization()
        self.conv1 = tf.keras.layers.Conv2D(filter_out, kernel_size, padding='same')
        
        self.bn2 = tf.keras.layers.BatchNormalization()
        self.conv2 = tf.keras.layers.Conv2D(filter_out, kernel_size, padding='same')
        
        if filter_in == filter_out:
            self.identity = lambda x : x
        else:
            self.identity = tf.keras.layers.Conv2D(filter_out, (1, 1), padding='same')
        
    def call(self, x, training=False, mask=None):
        h = self.bn1(x, training=training) # 배치 정규화
        h = tf.nn.relu(h)
        h = self.conv1(h)
        
        h = self.bn2(h, training=training)
        h = tf.nn.relu(h)
        h = self.conv2(h)
        return self.identity(x) + h

## Residual Layer 구현

In [4]:
class ResnetLayer(tf.keras.Model):
    def __init__(self, filter_in, filters, kernel_size):
        super(ResnetLayer, self).__init__()
        # 16, [32, 32, 32, 32] <-- Residual Unit 연결 형태
        self.sequence = list()
        for f_in, f_out in zip([filter_in] + list(filters), filters): 
            # 처음 filter_in = [16] + [32, 32, 32]
            # zip([16, 32, 32, 32], [32, 32, 32])
            # for문 돌리면 ==> [16, 32], [32, 32], [32, 32] ==> 나머지 32는 무시한다.
            self.sequence.append(ResidualUnit(f_in, f_out, kernel_size))
        
    def call(self, x, training=False, mask=None):
        for unit in self.sequence:
            # 주의 배치 정규화를 하였고 training을 넘겨 받았기 때문에 training도 넣어줘야함
            x = unit(x, training = training)
        return x
            

## 모델 정의

In [5]:
class ResNet(tf.keras.Model):
    def __init__(self):
        super(ResNet, self).__init__()
        self.conv1 = tf.keras.layers.Conv2D(8, (3, 3), padding='same', activation='relu') # 28x28x28
        
        self.res1 = ResnetLayer(8, (16, 16), (3, 3)) # 첫번째 Resnet : 28x28x16
        self.pool1 = tf.keras.layers.MaxPool2D((2, 2)) # 14x14x16
        
        self.res2 = ResnetLayer(16, (32, 32), (3, 3)) # 두번째 Resnet : 14x14x32
        self.pool2 = tf.keras.layers.MaxPool2D((2, 2)) # 7x7x32
        
        self.res3 = ResnetLayer(32, (64, 64), (3, 3)) # 세번째 Resnet : 7x7x64
        self.flatten = tf.keras.layers.Flatten()
        self.dense1 = tf.keras.layers.Dense(128, activation='relu')
        self.dense2 = tf.keras.layers.Dense(10, activation='softmax')
        
    def call(self, x, training=False, mask=None):
        x = self.conv1(x)
        x = self.res1(x, training=training)
        x = self.pool1(x)
        x = self.res2(x, training=training)
        x = self.pool2(x)
        x = self.res3(x, training=training)
        x = self.flatten(x)
        x = self.dense1(x)
        return self.dense2(x)
        

## 학습, 테스트 루프 정의

In [6]:
# Implement training loop
@tf.function
def train_step(model, images, labels, loss_object, optimizer, train_loss, train_accuracy):
    with tf.GradientTape() as tape:
        predictions = model(images, training=True)
        loss = loss_object(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)

    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    train_loss(loss)
    train_accuracy(labels, predictions)

# Implement algorithm test
@tf.function
def test_step(model, images, labels, loss_object, test_loss, test_accuracy):
    predictions = model(images, training=False)

    t_loss = loss_object(labels, predictions)
    test_loss(t_loss)
    test_accuracy(labels, predictions)

##  데이터셋 준비

In [7]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

x_train = x_train[..., tf.newaxis].astype(np.float32)
x_test = x_test[..., tf.newaxis].astype(np.float32)

train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

## 학습 환경 정의

#### 모델 생성, 손실 함수, 최적화 알고리즘, 평가지표 정의

In [8]:
# Create model
model = ResNet()

# Define loss and optimizer
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

# Define performance metrics
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

## 학습 루프 동작

In [10]:
for epoch in range(EPOCHS):
    for images, labels in train_ds:
        train_step(model, images, labels, loss_object, optimizer, train_loss, train_accuracy)

    for test_images, test_labels in test_ds:
        test_step(model, test_images, test_labels, loss_object, test_loss, test_accuracy)

    template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
    print(template.format(epoch + 1,
                          train_loss.result(),
                          train_accuracy.result() * 100,
                          test_loss.result(),
                          test_accuracy.result() * 100))
    train_loss.reset_states()
    train_accuracy.reset_states()
    test_loss.reset_states()
    test_accuracy.reset_states()

Epoch 1, Loss: 0.016634352505207062, Accuracy: 99.538330078125, Test Loss: 0.04785677790641785, Test Accuracy: 98.9000015258789
Epoch 2, Loss: 0.019516270607709885, Accuracy: 99.46833038330078, Test Loss: 0.0512625053524971, Test Accuracy: 98.86000061035156
Epoch 3, Loss: 0.016887696459889412, Accuracy: 99.57500457763672, Test Loss: 0.04766734689474106, Test Accuracy: 99.0999984741211
Epoch 4, Loss: 0.018137391656637192, Accuracy: 99.54000091552734, Test Loss: 0.06183047965168953, Test Accuracy: 98.6500015258789
Epoch 5, Loss: 0.01342307310551405, Accuracy: 99.6383285522461, Test Loss: 0.06666214764118195, Test Accuracy: 99.01000213623047
Epoch 6, Loss: 0.017725825309753418, Accuracy: 99.57833099365234, Test Loss: 0.0507744625210762, Test Accuracy: 98.95999908447266
Epoch 7, Loss: 0.01353498362004757, Accuracy: 99.66500091552734, Test Loss: 0.04065285623073578, Test Accuracy: 99.25
Epoch 8, Loss: 0.013409368693828583, Accuracy: 99.65499877929688, Test Loss: 0.04616790637373924, Test Ac

 -  중간중간에 오버 피팅이 발생하네...? ㄷㄷ

 - 확실히 그냥 Vanilla CNN보단 정확도가 빨리 증가한다